### 실시간 랭킹 데이터 수집

#### Package Import

In [4]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request, urlretrieve
from urllib.request import Request
from urllib.parse import quote_plus
from PIL import Image

#### 카테고리 ID 식별
- 랭킹차트에서 세부 카테고리를 선택하면, 페이지 요청 시 "mainCategory" 파라미터에, 값이 전달된다.
- 각 카테고리별 mainCategory param 값을 확인하여 request 하기.

In [5]:
# 
main_cat = ['전체', 
           '상의', '아우터', '바지', '원피스', '스커트', '스니커즈', '신발', '가방', '여성가방','스포츠용품', 
           '모자', '양말레그웨어', '속옷', '선글라스안경테', '액세서리', '시계', '주얼리', '뷰티', '디지털테크', '리빙',
           '컬쳐', '반려동물']

main_cat_id = ['', 
               '001', '002', '003', '020', '022', '018', '005', '004', '054', '017',
               '007', '008', '026', '009', '011', '006', '025', '015', '012', '058',
               '014', '021']

main_cat_dict = {}

for i in range(len(main_cat)):
  main_cat_dict[main_cat[i]] = main_cat_id[i]

# main_cat_dict

In [6]:
# 카테고리 확인
# for i in main_cat_dict.keys():
# print(i)

In [7]:
base_url_01 = 'https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory='
base_url_02 = '&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=False&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&includeWomen=false&priceMin=&priceMax='
search_url = base_url_01 + main_cat_dict['아우터'] + base_url_02

# search_url

#### 카테고리별 상품 랭킹 리스트 - 세부 상품정보 url 가져오기
- 랭킹 페이지에서 보이는 상품들의 '세부 상품정보 href (상품 url)' 값 가져오기

In [8]:
html = Request(search_url, headers={'User-Agent' : 'Mozilla/5.0'})
page = urlopen(html)
# print(page)

selector = '#goodsRankList > li div.li_inner > div.article_info > p.list_info > a'
soup = bs(page, 'html.parser')
selected = soup.select(selector)

items_url = []
for item in selected:
    items_url.append(item['href'])

# print(len(items_url), items_url, sep='\n')

#### 상품 기본 정보 가져오기
- 상품 url : 랭킹리스트에서 클릭했을 때 이동하는 상품 세부 정보 페이지
  - 브랜드, 제품품번, 제품명(한글), 제품명(영어), 가격(비회원), 상품 좋아요 수 등의 정보는 상품 url 클릭 후 이동한 페이지에서 수집되어야 함.

In [9]:
selector1 = '#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong'
selector2 = '#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong > a'
selector3 = '#page_product_detail > div.right_area.page_detail_product > div.right_contents.section_product_summary > span > em'
selector4 = '#sPrice > ul > li.pertinent > span.txt_price_member'
selector4_2 = '#product_order_info > div.explan_product.price_info_section > ul > li:nth-child(3) > div > span.product_article_price.product_article_price--clearance'
selector5 = '#product-top-like > p.product_article_contents.goods_like_3453194 > span'

In [10]:
product_id = []
brand_name = []
product_title = []
product_price = []
# product_like = []

for url in items_url:
  html = Request(url, headers={'User-Agent' : 'Mozilla/5.0'})
  page = urlopen(html)
  soup = bs(page, 'html.parser')  
  
  selected01 = soup.select(selector1)
  selected02 = soup.select(selector2)
  selected03 = soup.select(selector3)
  selected04 = soup.select(selector4)
  selected04_2 = soup.select(selector4_2)
  # selected05 = soup.select(selector5)

  if not selected04:
    selected04 = selected04_2

#  for pid, name, title, price, like in zip(selected01, selected02, selected03, selected04, selected05):

for pid, name, title, price in zip(selected01, selected02, selected03, selected04):
    product_id.append(pid.get_text().split(' / ')[1])
    brand_name.append(name.get_text())
    product_title.append(title.get_text().split(' / ')[0])
    product_price.append(price.get_text().replace(',', '').split('원')[0])
    # product_like.append(like.get_text().replace(',', ''))

KeyboardInterrupt: 

In [ ]:
# print(len(product_id), len(brand_name), len(product_title), len(product_price), len(product_like))
print(len(product_id), len(brand_name), len(product_title), len(product_price))

90 90 90 90


In [ ]:
# pd.DataFrame({'품번' : product_id, '브랜드명' : brand_name, '상품명(한글)' : product_title ,
#              '가격' : product_price , '좋아요 수' : product_like})

pd.DataFrame({'품번' : product_id, '브랜드명' : brand_name, '상품명(한글)' : product_title ,
              '가격' : product_price})

,품번,브랜드명,상품명(한글),가격
0,T1S-2WHZU03,TOFFEE,2WAY 스웻 후드 집업 (MELANGE GREY),39000
1,MG2300HZ01GR,WHATITISNT,엔젤 와펜 후드집업 그레이,47400
2,K23AWUCD02BL,KNITTED,쉐기독 울 100% 라운드 가디건 BLUE,98900
3,B-OUTER-0033-OLIVE,BRONSON,US Air Force Type L-2 Flight Jacket Olive,215000
4,WBC3L04503GR,GENERALIDEA,GNRL 에센셜 가디건 [GREY],49000
...,...,...,...,...
85,MG2300HZ01OT,WHATITISNT,엔젤 와펜 후드집업 오트밀,47400
86,1649328,WV PROJECT,와핑 후드집업 그레이 JJOT7203,39800
87,YC8HD0003NA,YALE,(23FW) [ONEMILE WEAR] SMALL ARCH LOGO HOODIE Z...,69000
88,NJ3NP85D,THE NORTH FACE,NJ3NP85D 여성 눕시 온볼 자켓,249000


In [ ]:
from datetime import datetime

In [ ]:
########### 카테고리별 상품 랭킹 리스트 
for key, value in main_cat_dict.items():
    
    base_url_01 = 'https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory='
    base_url_02 = '&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=False&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&includeWomen=false&priceMin=&priceMax='
    search_url = base_url_01 + value + base_url_02

    html = Request(search_url, headers={'User-Agent' : 'Mozilla/5.0'})
    page = urlopen(html)
    # print(page)
    
    selector = '#goodsRankList > li div.li_inner > div.article_info > p.list_info > a'
    soup = bs(page, 'html.parser')
    selected = soup.select(selector)

    items_url = []
    for item in selected:
        items_url.append(item['href'])

    product_id = []
    brand_name = []
    product_title = []
    product_price = []
    # product_like = []

    for url in items_url:
        html = Request(url, headers={'User-Agent' : 'Mozilla/5.0'})
        page = urlopen(html)
        soup = bs(page, 'html.parser')  
        
        selected01 = soup.select(selector1)
        selected02 = soup.select(selector2)
        selected03 = soup.select(selector3)
        selected04 = soup.select(selector4)
        selected04_2 = soup.select(selector4_2)
        # selected05 = soup.select(selector5)

        if not selected04:
            selected04 = selected04_2

    #  for pid, name, title, price, like in zip(selected01, selected02, selected03, selected04, selected05):
        for pid, name, title, price in zip(selected01, selected02, selected03, selected04):
            product_id.append(pid.get_text().split(' / ')[1])
            brand_name.append(name.get_text())
            product_title.append(title.get_text().split(' / ')[0])
            product_price.append(price.get_text().replace(',', '').split('원')[0])
    
    df = pd.DataFrame({'품번' : product_id, '브랜드명' : brand_name, '상품명(한글)' : product_title ,
              '가격' : product_price})

    now = datetime.now()
    now_str = now.strftime("%Y%m%d")
    df.to_csv(f'무신사_남녀통합_{key}_Top90_{now_str}.csv', index=False, encoding='utf-8-sig')